In [47]:
import yfinance as yf 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 
import os


from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

from tensorflow import keras
Sequential = keras.models.Sequential
LSTM = keras.layers.LSTM
Dense = keras.layers.Dense
Dropout = keras.layers.Dropout
EarlyStopping = keras.callbacks.EarlyStopping
callbacks = keras.callbacks



In [61]:
scaler = MinMaxScaler(feature_range=(0,1))
def crypto(data, interval = "1h"):
    data_complete = yf.download(data, interval = interval)
    data = scaler.fit_transform(data_complete['Adj Close'])
    return data, data_complete

def create_sequence(data, sequence=60):
    X, y = [], []
    for i in range(len(data)- sequence):
        X.append(data[i:i + sequence])
        y.append(data[i + sequence])
    return np.array(X), np.array(y)

def trainning(data):
    X, y = create_sequence(data=data)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42 )
    
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1 )

    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')
    early_stopping = EarlyStopping( patience=2)

    model.fit(X_train, 
            y_train, 
            epochs = 1, 
            batch_size = 32,
            validation_data=(X_test, y_test),
            verbose=1, 
            callbacks = [early_stopping])

    y_pred = model.predict(X_test)

    y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1,1))
    y_pred_rescaled = scaler.inverse_transform(y_pred)

    return model, y_test_rescaled, y_pred_rescaled, X_test, y_test


def metrics_model(y_test_rescaled, y_pred_rescaled):
    rmse = np.sqrt(mean_squared_error(y_test_rescaled, y_pred_rescaled))
    mae = mean_absolute_error(y_test_rescaled, y_pred_rescaled)
    r2 = r2_score(y_test_rescaled, y_pred_rescaled)
    return [rmse, mae, r2]

def historic_graphic(data_complete,y_test_rescaled, y_pred_rescaled):
    prediction_dates = data_complete[-len(y_test_rescaled):]
    plt.figure(figsize=(10,6))
    plt.plot(prediction_dates, y_test_rescaled, label='Actual', c='r')
    plt.plot(prediction_dates, y_pred_rescaled, label='Prediccetd', c='b')
    plt.xlabel('Date')
    plt.ylabel('Price USD')
    plt.title('Comparative betwen time and price')
    plt.legend()
    plt.show()


    

def predict_next_week(model, data, num_days, sequence=60):
    last_sequence = data[-sequence:].reshape(1, sequence, 1)
    predicted_prices = []
    for _ in range(num_days):
        predicted_price = model.predict(last_sequence)
        predicted_prices.append(predicted_price)
        last_sequence = np.append(last_sequence[:, 1:, :], predicted_price.reshape(1,1,1), axis=1)
    predicted_prices_rescaled = scaler.inverse_transform(np.array(predicted_prices).reshape(-1,1))

    return predicted_prices_rescaled

In [ ]:
cryptos = ['DOGE-USD', 'BTC-USD', 'ETH-USD', 'TSL-USD']
for i in range(len(cryptos)):
    os.mkdir(path=f'./Graphcis{cryptos[i]}')

names_complete = [f'{i}_complete' for i in cryptos]
data_1 = [f'{i}_scaler' for i in cryptos]

df_complete = {}
df_data = {}
models = {}
metrics = {}



for i in range(len(cryptos)):
    df_data[cryptos[i]], df_complete[cryptos[i]] = crypto(data=cryptos[i], interval="1d")
    models[f'model_{cryptos[i]}'], y_test_rescaled, y_pred_rescaled, X_test, y_test = trainning(df_data[cryptos[i]])
    metrics[f'metrics_{cryptos[i]}'] = metrics_model(y_test_rescaled, y_pred_rescaled)


[*********************100%***********************]  1 of 1 completed
/home/lic-jimenez/anaconda3/envs/mi_entorno/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


64/64 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - loss: 0.0076 - val_loss: 0.0014
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


[*********************100%***********************]  1 of 1 completed
/home/lic-jimenez/anaconda3/envs/mi_entorno/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


92/92 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 0.0132 - val_loss: 5.1052e-04
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step


[*********************100%***********************]  1 of 1 completed
/home/lic-jimenez/anaconda3/envs/mi_entorno/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


64/64 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - loss: 0.0473 - val_loss: 0.0015
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step


[*********************100%***********************]  1 of 1 completed
/home/lic-jimenez/anaconda3/envs/mi_entorno/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - loss: 3.3542e-04 - val_loss: 5.7353e-05
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step


In [60]:
for i in df_data:
    print(i)

DOGE-USD
BTC-USD
ETH-USD
TSL-USD
